In [1]:
import os, sys, json
cwd = os.getcwd()
PROJECT_PATH = cwd[:cwd.find('midfreq')+7]
print('PROJECT_PATH:', PROJECT_PATH)
if PROJECT_PATH not in sys.path: sys.path.append(PROJECT_PATH)
os.chdir(PROJECT_PATH) 

from datetime import datetime, timedelta, timezone
import pandas as pd

PROJECT_PATH: /home/user/singularity/midfreq


In [2]:
import plotly.express as px
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# read files

In [3]:
# vault = 'DynYieldB'
vault = 'DynYieldE'
dirname = os.path.join(os.sep, os.getcwd(), 'data', vault)
pools = []
for filename in os.listdir(dirname):
    fullname = os.path.join(os.sep, dirname, filename)
    print(fullname)
    data = pd.read_csv(fullname, parse_dates=['date']).set_index('date')['haircut_apy']
    pools.append(data.rename(filename.replace('.csv','')))
history = pd.concat(pools, axis=1)

/home/user/singularity/midfreq/data/DynYieldE/325ad2d6-70b1-48d7-a557-c2c99a036f87.csv
/home/user/singularity/midfreq/data/DynYieldE/405d8dad-5c99-4c91-90d3-82813ade1ff1.csv
/home/user/singularity/midfreq/data/DynYieldE/849e0897-1d5c-4cbb-afc9-289e0659e483.csv
/home/user/singularity/midfreq/data/DynYieldE/d536e3f6-adf1-4364-afc9-ee8755b81d98.csv
/home/user/singularity/midfreq/data/DynYieldE/40e2054c-779d-4499-845f-95b4a31277f7.csv
/home/user/singularity/midfreq/data/DynYieldE/efbf1a35-0756-41b1-b230-166ed4809314.csv
/home/user/singularity/midfreq/data/DynYieldE/1343a280-7812-4bc3-8f98-d1c37e11d271.csv
/home/user/singularity/midfreq/data/DynYieldE/0b65cb9f-2c63-40d5-9344-2a867fbfc41a.csv
/home/user/singularity/midfreq/data/DynYieldE/3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1.csv
/home/user/singularity/midfreq/data/DynYieldE/26298be5-691a-42e6-817b-07f72e70cd4f.csv
/home/user/singularity/midfreq/data/DynYieldE/de91ccee-cb64-43be-b1d8-124f79ef62a0.csv
/home/user/singularity/midfreq/data/DynYiel

In [4]:
# metadata = pd.read_csv(f"C:/Users/david/singularity/midfreq/scrappers/defillama_history/data/{vault}_pool_metadata.csv")
metadata = pd.read_csv(f"{PROJECT_PATH}/data/{vault}_pool_metadata.csv")
pool_map = metadata[['pool','project','symbol']].set_index('pool').T.to_dict()
pool_map = {key: '{}_{}'.format(x['project'], x['symbol']) for key, x in pool_map.items()}
index_map = {f'weight_{i}': f'w_{value}' for i, value in enumerate(pool_map.values())}
index_map |= {f'yield_{i}': f'y_{value}' for i, value in enumerate(pool_map.values())} 
history.columns = [pool_map[col] for col in history.columns]

In [5]:
pool_map

{'c8a24fee-ec00-4f38-86c0-9f6daebc4225': 'makerdao_DAI',
 'ccc467d1-bdf5-4062-81cd-7aed7dd9050b': 'makerdao_WSTETH',
 '51d2f8d4-1fb5-4f6b-938b-e9cd17ca1ceb': 'makerdao_WETH',
 '54600482-cfb9-4dd9-a73b-342cffd4644a': 'makerdao_WETH',
 '51eb4b5c-6d62-42fe-99d6-92ba6eecd2d5': 'makerdao_WSTETH',
 '051c1a54-df05-4bc4-9d73-c17508acae23': 'makerdao_USDC',
 '25171c4c-1877-449a-9f88-45a9f153ee31': 'curve-dex_DAI-USDC-USDT',
 'bd335b46-3daf-4507-adb2-6fd4337ea33b': 'makerdao_WBTC',
 '44c94cb2-8af3-4b7a-8cb5-0784ddcf3471': 'makerdao_WETH',
 '3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1': 'curve-dex_FRAX-USDC',
 '6ece81c6-2b1d-4513-ad1c-eaf9d52f67cb': 'makerdao_PAX',
 '4dab1166-614a-429a-831f-4af70fd29c7d': 'makerdao_WBTC',
 'efbf1a35-0756-41b1-b230-166ed4809314': 'makerdao_GUSD',
 '1193ef25-862b-43c1-a545-91bbb9678d30': 'uniswap-v3_DAI-USDC',
 'f38a1e55-ee6a-427c-83b5-74232e1b2c24': 'makerdao_G-UNI',
 'd371fc69-9a77-4336-a129-ec79c9548266': 'uniswap-v3_FRAX-USDC',
 'bd072651-d99c-4154-aeae-51f12109c054':

# compute historical pool rank

In [6]:
max_rank = 10
end = datetime.now().replace(tzinfo=timezone.utc)
start = end - timedelta(days = 90)
df = history.fillna(method='ffill')
df = df[(df.index >= start)&(df.index <= end)]
best_only = df[df.rank(axis=1, ascending=False)<max_rank].dropna(how='all', axis=1)
#best_only = best_only.rename(columns=pool_map)
best_only.iplot()
best_only

/tmp/ipykernel_134189/1917900268.py:4: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



,morpho-aave_USDC,aave-v2_DAI,curve-dex_FRAX-CRVUSD,convex-finance_MKUSD-FRAXBP,uniswap-v3_USDC-USDT,morpho-aave_USDT,curve-dex_FRAX-USDC,uniswap-v3_LUSD-USDC,curve-dex_CDAI-CUSDC,uniswap-v3_DAI-USDT,...,convex-finance_FRAX-CRVUSD,aave-v3_USDC,curve-dex_USDT-CRVUSD,curve-dex_ADAI-AUSDC-AUSDT,curve-dex_CRVUSD-SDAI,convex-finance_USDC-CRVUSD,aave-v3_DAI,aura_R-SDAI,aave-v3_USDT,aave-v2_USDT
date,,,,,,,,,,,,,,,,,,,,,
2023-08-08 00:00:00+00:00,NaN,0.072932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.070786,NaN,0.070853,NaN,NaN,0.078294,NaN,NaN,NaN,NaN
2023-08-09 00:00:00+00:00,0.095493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.082776,...,NaN,0.088831,NaN,NaN,NaN,NaN,0.124573,NaN,NaN,NaN
2023-08-10 00:00:00+00:00,0.094810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.072470,0.078911,NaN,NaN,0.071925,NaN,NaN,NaN,NaN
2023-08-11 00:00:00+00:00,0.137833,NaN,NaN,NaN,NaN,0.221210,NaN,NaN,NaN,NaN,...,NaN,0.097094,NaN,NaN,NaN,NaN,NaN,NaN,0.099334,0.160935
2023-08-12 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.086546,NaN,0.070377,0.108984,NaN,0.068069,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-31 00:00:00+00:00,NaN,NaN,0.080976,NaN,NaN,0.077327,NaN,0.091469,NaN,0.142229,...,0.119439,NaN,NaN,NaN,NaN,NaN,NaN,0.147563,NaN,NaN
2023-11-01 00:00:00+00:00,NaN,NaN,0.087692,NaN,NaN,0.081892,NaN,0.258268,NaN,0.102400,...,0.127974,NaN,NaN,NaN,NaN,NaN,NaN,0.212640,NaN,NaN
2023-11-02 00:00:00+00:00,NaN,NaN,NaN,0.223686,NaN,NaN,NaN,0.183404,NaN,NaN,...,0.136215,NaN,NaN,NaN,NaN,NaN,NaN,0.218982,NaN,NaN


In [7]:
pool_map

{'c8a24fee-ec00-4f38-86c0-9f6daebc4225': 'makerdao_DAI',
 'ccc467d1-bdf5-4062-81cd-7aed7dd9050b': 'makerdao_WSTETH',
 '51d2f8d4-1fb5-4f6b-938b-e9cd17ca1ceb': 'makerdao_WETH',
 '54600482-cfb9-4dd9-a73b-342cffd4644a': 'makerdao_WETH',
 '51eb4b5c-6d62-42fe-99d6-92ba6eecd2d5': 'makerdao_WSTETH',
 '051c1a54-df05-4bc4-9d73-c17508acae23': 'makerdao_USDC',
 '25171c4c-1877-449a-9f88-45a9f153ee31': 'curve-dex_DAI-USDC-USDT',
 'bd335b46-3daf-4507-adb2-6fd4337ea33b': 'makerdao_WBTC',
 '44c94cb2-8af3-4b7a-8cb5-0784ddcf3471': 'makerdao_WETH',
 '3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1': 'curve-dex_FRAX-USDC',
 '6ece81c6-2b1d-4513-ad1c-eaf9d52f67cb': 'makerdao_PAX',
 '4dab1166-614a-429a-831f-4af70fd29c7d': 'makerdao_WBTC',
 'efbf1a35-0756-41b1-b230-166ed4809314': 'makerdao_GUSD',
 '1193ef25-862b-43c1-a545-91bbb9678d30': 'uniswap-v3_DAI-USDC',
 'f38a1e55-ee6a-427c-83b5-74232e1b2c24': 'makerdao_G-UNI',
 'd371fc69-9a77-4336-a129-ec79c9548266': 'uniswap-v3_FRAX-USDC',
 'bd072651-d99c-4154-aeae-51f12109c054':

In [8]:
for descr in best_only.columns:
    poolId = next((key for key, value in pool_map.items() if value == descr), [])
    print(f'- \"{poolId}\" # {descr}')

- "325ad2d6-70b1-48d7-a557-c2c99a036f87" # morpho-aave_USDC
- "405d8dad-5c99-4c91-90d3-82813ade1ff1" # aave-v2_DAI
- "849e0897-1d5c-4cbb-afc9-289e0659e483" # curve-dex_FRAX-CRVUSD
- "d536e3f6-adf1-4364-afc9-ee8755b81d98" # convex-finance_MKUSD-FRAXBP
- "e737d721-f45c-40f0-9793-9f56261862b9" # uniswap-v3_USDC-USDT
- "1343a280-7812-4bc3-8f98-d1c37e11d271" # morpho-aave_USDT
- "3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1" # curve-dex_FRAX-USDC
- "de91ccee-cb64-43be-b1d8-124f79ef62a0" # uniswap-v3_LUSD-USDC
- "8ec9e3e2-1eb5-4c47-82de-edcc825ca4cc" # curve-dex_CDAI-CUSDC
- "802d35a2-5bae-456f-a2a9-4e1b48e0dec5" # uniswap-v3_DAI-USDT
- "a3ffd3fe-b21c-44eb-94d5-22c80057a600" # convex-finance_USDT-CRVUSD
- "57647093-2868-4e65-97ab-9cae8ec74e7d" # compound_USDT
- "a349fea4-d780-4e16-973e-70ca9b606db2" # aave-v2_USDC
- "bd072651-d99c-4154-aeae-51f12109c054" # convex-finance_FRAX-USDC
- "c8a24fee-ec00-4f38-86c0-9f6daebc4225" # makerdao_DAI
- "15cbe783-dc40-4533-aa60-94b8d338aa7c" # curve-dex_SUSD-CRVUSD

In [9]:
from scrappers.defillama_history.defillama import DynLst, DynYieldE, DynYieldB
short_listed_protocols = sum([vault.getattr(short_listed_protocols) for vault in [DynLst, DynYieldE, DynYieldB]])

ModuleNotFoundError: No module named 'retry'